Reorganize the two output csv (hyperlink approach and google search approach)
Into a format that is easy to be plotted into SNA Plot

Best way is into format of pairs (X, Y)

# Combine the results

In [1]:
# Regular Python Data analyze library
import pandas as pd
import random
import math

In [ ]:
# Load CSV into a Data Pool DataFrame
# Change encoding from 'utf-8' to 'latin1' to avoid invalid continuation byte
df_pool = pd.read_csv('')

# Apply .strip() to all string values in the DataFrame using apply with a lambda function
df_pool = df_pool.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))

# Load CSV into a Data Pool DataFrame
# Change encoding from 'utf-8' to 'latin1' to avoid invalid continuation byte
df_hyper = pd.read_csv('search_results_Hyperlink.csv', encoding='latin1')

# Apply .strip() to all string values in the DataFrame using apply with a lambda function
df_hyper = df_hyper.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))

# Load CSV into a Data Pool DataFrame
# Change encoding from 'utf-8' to 'latin1' to avoid invalid continuation byte
df_google = pd.read_csv('search_results_web_search_1.csv', encoding='latin1')

# Apply .strip() to all string values in the DataFrame using apply with a lambda function
df_google = df_google.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))

In [3]:
df_target = df_pool
# We dont need any duplications
df_target = df_target.drop_duplicates(subset='Sponsors')
df_target.reset_index(drop=True, inplace=True)
Sponsor_list = df_target['Sponsors'].tolist()
print("Unique number of Sponsors:", len(Sponsor_list))
Sponsor_combinations = []
for i in range(len(Sponsor_list)):
  for j in range(i + 1, len(Sponsor_list)):
    Sponsor_combinations.append(frozenset([Sponsor_list[i], Sponsor_list[j]]))
print("Number of Sponsor Combinations:", len(Sponsor_combinations))

Unique number of Sponsors: 84
Number of Sponsor Combinations: 3486


In [4]:
# Helper function to check NAN
def is_nan(value):
  """Check if the given value is NaN."""
  return isinstance(value, float) and math.isnan(value)

In [5]:
# Process through hyperlink
# Remove all rows that "Matched External Link" is nan
df_hyper = df_hyper[df_hyper['Matched External Link'].apply(lambda x: not is_nan(x))]
# Reset Index
df_hyper = df_hyper.reset_index(drop=True)

In [6]:
# Loop through df_hyper
# Create a pair using (Homepage Sponsor, Target Sponsor)
# Use set of this pair as key (so (Home, Target) will be treated the same as (Target, Home))
# And Matched External Link as value, create a map

# Now you can run the loop for creating the pair_map
pair_map_hyper = {}

for _, row in df_hyper.iterrows():
  # Create a set using 'Homepage Sponsor' and 'Target Sponsor' to ensure order doesn't matter
  # Use frozenset to ensure set is inmutable thus can be used as key
  sponsor_pair = frozenset([row['Homepage Sponsor'], row['Target Sponsor']])

  # Add to the dictionary, using the set as key and the 'Matched External Link' as value
  pair_map_hyper[sponsor_pair] = row['Matched External Link']

In [7]:
# Loop through df_google
# Create a pair using (Sponsor 1, Sponsor 2)
# Use set of this pair as key (so (Sponsor 1, Sponsor 2) will be treated the same as (Sponsor 2, Sponsor 1))
# And Matched External Link as value, create a map

# Now you can run the loop for creating the pair_map
pair_map_google = {}

for _, row in df_google.iterrows():
  # Create a set using 'Homepage Sponsor' and 'Target Sponsor' to ensure order doesn't matter
  # Use frozenset to ensure set is inmutable thus can be used as key
  sponsor_pair = frozenset([row['Sponsor 1'], row['Sponsor 2']])

  # Add to the dictionary, using the set as key and the 'Matched External Link' as value
  pair_map_google[sponsor_pair] = row['Representing Result']

In [8]:
# list to store final result
final_results = []
# Loop through all combinations and make the final dataframe
for pair in Sponsor_combinations:
  # if pair in both dictionary
  if pair in pair_map_hyper and pair in pair_map_google:
    final_results.append({
      "Edge": tuple(pair),
      "Google_proof": pair_map_google[pair],
      "Hyperlink_proof": pair_map_hyper[pair]
    })
  # if pair only in one dictionary
  elif pair in pair_map_hyper and not pair in pair_map_google:
    final_results.append({
      "Edge": tuple(pair),
      "Google_proof": "None",
      "Hyperlink_proof": pair_map_hyper[pair]
    })
  elif pair not in pair_map_hyper and pair in pair_map_google:
    final_results.append({
      "Edge": tuple(pair),
      "Google_proof": pair_map_google[pair],
      "Hyperlink_proof": "None"
    })
  else:
    continue

In [9]:
df_results = pd.DataFrame(final_results)
df_results.tail()

,Edge,Google_proof,Hyperlink_proof
546,"(U.S. Small Business Administration, Wells Fargo)",SearchResult(url=https://www.wellsfargo.com/bi...,https://www.sba.gov/funding-programs/loans/cov...
547,"(U.S. Small Business Administration, Women's E...",SearchResult(url=https://wedcbiz.org/resources...,https://www.sba.gov/
548,"(Valley National Bank, Union Bank)",SearchResult(url=https://www.valley.com/about/...,None
549,"(Women's Enterprise Development Center, Verizo...",None,https://digitalready.verizonwireless.com/?utm_...
550,"(Wells Fargo, Women's Enterprise Development C...",SearchResult(url=https://wedcbiz.org/about-us/...,None


In [10]:
# Output result
df_results.to_csv('final_results.csv', index=False)

In [11]:
# Expand final_results
# For each partner in the partner list, create an edge with original target corporate
target_corporate = "Lead B"

for partner in Sponsor_list:
  # Create a pair
  # This pair should not be included in the previous list at all
  pair = (target_corporate, partner)
  final_results.append({
    "Edge": tuple(pair),
    "Google_proof": "None",
    "Hyperlink_proof": "None"
  })

In [12]:
# Convert to a new dataframe
df_results_expand = pd.DataFrame(final_results)
df_results_expand.tail()

,Edge,Google_proof,Hyperlink_proof
630,"(Lead B, Wells Fargo)",None,None
631,"(Lead B, West Brighton Community LDC)",None,None
632,"(Lead B, Whistler Capital Partners)",None,None
633,"(Lead B, Women's Enterprise Development Center)",None,None
634,"(Lead B, YMCA of Brooklyn)",None,None


In [13]:
# Output expanded result
df_results_expand.to_csv('masked_final_results_expand.csv', index=False)